In [1]:
import nltk
from nltk.tokenize import word_tokenize
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot') #Setting style sheets that we're using for plot
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
calls = pd.read_csv('calls.csv')
customers = pd.read_csv('customers.csv')
reason = pd.read_csv('reason.csv')
sentiment = pd.read_csv('sentiment_statisticsc.csv')
test = pd.read_csv('test.csv')

In [17]:
reason.head()

,call_id,primary_call_reason
0,4667960400,Voluntary Cancel
1,1122072124,Booking
2,6834291559,IRROPS
3,2266439882,Upgrade
4,1211603231,Seating


In [20]:
reason['primary_call_reason'].value_counts()

IRROPS                     13057
Voluntary Change           10291
Seating                     6223
Mileage Plus                5487
Post-Flight                 3869
Communications              3779
Products and Services       2792
Upgrade                     2682
Baggage                     2616
Booking                     2589
Checkout                    1840
Check-In                    1490
Voluntary Cancel            1304
Digital Support              996
ETC                          930
Traveler Updates             772
Schedule Change              707
Other Topics                 568
Products & Services          476
Disability                   394
Check In                     385
Post Flight                  373
Voluntary  Cancel            278
IRROPS                       254
Other  Topics                234
Voluntary Change             208
Voluntary   Change           200
Voluntary  Change            149
Seating                      142
Digital   Support            136
Mileage  P

In [10]:
sentiment.head()


,call_id,agent_id,agent_tone,customer_tone,average_sentiment,silence_percent_average
0,4667960400,963118,neutral,angry,-0.04,0.39
1,1122072124,519057,calm,neutral,0.02,0.35
2,6834291559,158319,neutral,polite,-0.13,0.32
3,2266439882,488324,neutral,frustrated,-0.20,0.20
4,1211603231,721730,neutral,polite,-0.05,0.35


In [35]:
sentiment_df = sentiment.groupby('agent_tone')['average_sentiment'].mean().reset_index()
sentiment_df

,customer_tone,average_sentiment
0,angry,-0.032427
1,calm,-0.032501
2,frustrated,-0.034490
3,neutral,-0.033709
4,polite,-0.034475


In [31]:
sentiment['agent_tone'].value_counts()

neutral       42758
calm          24675
frustrated     3684
angry           389
polite           87
Name: agent_tone, dtype: int64

In [16]:
sentiment['customer_tone'].value_counts()

neutral       14444
angry         14408
polite        14371
calm          14347
frustrated    14240
Name: customer_tone, dtype: int64

In [34]:
calls_shape = calls.shape
customers_shape = customers.shape
reason_shape = reason.shape
sentiment_shape = sentiment.shape

In [9]:
print(calls_shape)
print(customers_shape)
print(reason_shape)
print(sentiment_shape)

(71810, 7)
(71810, 3)
(66653, 2)
(71810, 6)


In [11]:
calls.tail()

,call_id,customer_id,agent_id,call_start_datetime,agent_assigned_datetime,call_end_datetime,call_transcript
71805,1563273072,8022667294,413148,8/31/2024 23:48,8/31/2024 23:57,9/1/2024 0:25,\n\nAgent: Thank you for calling United Airlin...
71806,8865997781,4474867021,980156,8/31/2024 23:55,8/31/2024 23:58,9/1/2024 0:06,\n\nAgent: Thank you for calling United Airlin...
71807,8019240181,9762042472,616988,8/31/2024 23:52,8/31/2024 23:58,9/1/2024 0:04,\n\nAgent: Thank you for calling United Airlin...
71808,8332067080,8095620266,161354,8/31/2024 23:53,8/31/2024 23:59,9/1/2024 0:03,\n\nAgent: Thank you for calling United Airlin...
71809,8210720833,8014388758,547592,8/31/2024 23:49,8/31/2024 23:59,9/1/2024 0:02,\n\nAgent: Thank you for calling United Airlin...


In [13]:
sentiment.tail()

,call_id,agent_id,agent_tone,customer_tone,average_sentiment,silence_percent_average
71805,1563273072,413148,calm,calm,0.05,0.34
71806,8865997781,980156,calm,frustrated,0.03,0.22
71807,8019240181,616988,calm,polite,0.05,0.42
71808,8332067080,161354,neutral,polite,-0.07,0.13
71809,8210720833,547592,neutral,calm,0.00,0.10


In [21]:
reason_df = reason

In [22]:
reason_df.head()

,call_id,primary_call_reason
0,4667960400,Voluntary Cancel
1,1122072124,Booking
2,6834291559,IRROPS
3,2266439882,Upgrade
4,1211603231,Seating


In [23]:
reason_df['primary_call_reason'] = (reason_df['primary_call_reason']
                .str.lower()                                  # Convert to lowercase
                .str.strip()                                  # Remove leading and trailing spaces
                .str.replace(r'\s+', ' ', regex=True))

In [26]:
reason_df['primary_call_reason'].value_counts()

irrops                   13311
voluntary change         10848
seating                   6365
mileage plus              5851
post-flight               3957
communications            3840
products and services     2856
baggage                   2832
upgrade                   2738
booking                   2637
checkout                  1888
voluntary cancel          1607
check-in                  1519
digital support           1225
etc                        952
traveler updates           937
other topics               818
schedule change            731
products & services        476
disability                 403
check in                   385
post flight                373
unaccompanied minor        104
Name: primary_call_reason, dtype: int64

In [28]:
pip install wordcloud

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [29]:
from wordcloud import WordCloud

In [36]:
# Convert the call_start_datetime and call_end_datetime columns to datetime format
calls = pd.to_datetime(calls['call_start_datetime'])
calls = pd.to_datetime(calls['call_end_datetime'])

# Calculate the time spent on each call
calls['call_duration'] = calls['call_end_datetime'] - calls['call_start_datetime']

# Display the DataFrame with the new call_duration column
print(calls[['call_id', 'call_start_datetime', 'call_end_datetime', 'call_duration']])

KeyError: 'call_end_datetime'